In [2]:
# imports
import pandas as pd
import numpy as np
import random
import statistics as stat

traindf = pd.read_pickle("./pickles/df_small_clean.pkl")

# display(traindf[traindf["position"] == 11])

In [3]:
def random_solution(df):
    
    # csv_name = "./results/" + csv_name
    
    # get list of hotels per search id
    results_per_search = df.groupby('srch_id').size()
    
    column_values = df["srch_id"].values
    
    ids =  pd.unique(column_values) 
    
    all_ranks = []
    
    for i, ideetje in enumerate(ids):
        df_temp = df[df["srch_id"] == ideetje]
        
        ranks = df_temp["position"].values
        
        # shuffle ranks
        random.shuffle(ranks)
        all_ranks.extend(ranks)

    df["predicted_rank"] = all_ranks
    return df

In [4]:
def indiv_search_ndcg_score(true_rank, predicted_rank, booked, clicked):
    # holder for ideal score
    idcg = 0
    
    # holder for our score
    dcg = 0

    # itereer over elk resultaat in de search
    for i in range(len(true_rank)):
        # hotels die geboekt zijn zijn heel belangrijk (5)
        if booked[i] == 1:
            
            # increment ideal score
            if i != 0:
                idcg = idcg + (5 / np.log2(i+1))
            else:
                idcg += 5

            # if predicted well, increment score
            if true_rank[i] == predicted_rank[i]:
                if i != 0:
                    dcg = dcg + (5 / np.log2(i+1))
                else:
                    dcg += 5
                
        # hotels waarop geklikt is zijn een beetje belangrijk (1)
        elif clicked[i] == 1:

            # increment ideal score
            if i != 0:
                idcg = idcg + (1 / np.log2(i+1))
            else:
                idcg += 1
            
            # if predicted well, increment score
            if true_rank[i] == predicted_rank[i]:
                dcg = dcg + (1 / np.log2(i+2))

    if idcg == 0:
        return dcg
    return dcg/idcg

In [17]:
# score function for every search (thus iterate over groupby object)
def dataframe_ndcg_score(solution):
    all_scores = []
    for srchid, group in solution.groupby('srch_id'):

        # true_rank
        true_rank = np.asarray(group['position'])

        # predicted_rank
        predicted_rank = np.asarray(group['predicted_rank'])
        
        # booking_bool
        booked = np.asarray(group['booking_bool'])
        
        # click_bool
        clicked = np.asarray(group['click_bool'])
        all_scores.append(indiv_search_ndcg_score(true_rank, predicted_rank, booked, clicked))
        
    return np.nanmean(all_scores)

In [21]:
for i in range(5):
    print(i, dataframe_ndcg_score(random_solution(traindf)))

0 0.034027418187044196
1 0.04783759203436864
2 0.06034697877359598
3 0.0496151807966268
4 0.04878483930678103


In [18]:
rf = pd.read_pickle("./pickles/random_forest_small_solution.pkl")

print(dataframe_ndcg_score(rf))

0.015544041450777202


In [19]:
rf.head(10)

,prop_id,srch_id,position,price_quality,competitor_bool,availability_bools,visited_before,click_bool,booking_bool,PriceBand,promotion_flag,prop_brand_bool,random_bool,predicted_rank
1597336,96260,107340,19,23.840000,0,1,0,0,0,6,1,0,0,36
3713106,65606,249343,7,43.980000,0,0,0,0,0,7,0,1,0,30
1645721,7459,110549,31,36.333332,1,0,0,0,0,2,0,1,0,29
4924736,92046,330548,13,19.610001,0,0,0,0,0,2,0,0,0,3
800009,110792,53512,27,27.000000,0,0,0,1,1,4,0,1,0,15
1620007,133007,108848,9,38.305000,0,1,0,0,0,6,0,0,0,8
4503513,101494,302149,9,52.333332,0,1,0,0,0,6,0,0,0,37
2392951,101052,160691,10,63.075001,1,1,0,0,0,7,0,0,0,14
1760153,4345,118200,3,59.666668,0,1,0,0,0,5,0,0,0,19
2582510,35615,173399,3,23.500000,0,0,0,0,0,3,0,1,0,9
